In [1]:
import torch

In [2]:
#devito set up
from abc import ABC, abstractmethod
from devito import Operator, Function
from numpy import array
import numpy as np
from devito import Grid, Function, dimensions, Eq, Inc
import sympy

In [3]:
batch_weights = torch.rand(2,3,3,3)

# python implementation

In [37]:
def batch_norm(weight):
    eps = 1e-5
    momentum = 0.1 #i dont know where it goes
    Var = torch.var(weight, unbiased=False, axis = (0,2,3))
    E = torch.mean(weight, axis = (0,2,3))
    print("E:", E, "Var",Var)
    gamma = 1
    betta = 0
    result = torch.zeros_like(batch_weights)
    for image in range(weight.shape[0]):
        for channel in range(weight.shape[1]):
            for x in range(weight.shape[2]):
                for y in range(weight.shape[3]):
                    result[image][channel][x][y]=(batch_weights[image][channel][x][y]-E[channel])\
                   /(np.sqrt(Var[channel]+1e-5))
    return result
py_res = batch_norm(batch_weights)

E: tensor([0.4423, 0.4008, 0.4268]) Var tensor([0.0629, 0.0745, 0.0931])


# pytorch implementation

In [5]:
m = torch.nn.BatchNorm2d(3, affine=False)
torch_output = m(batch_weights)

In [38]:
#compare
np.allclose(py_res, torch_output, atol = 1e-5)

True

# Devito implementation

Loops to find the variance and the mean. Consider implementing them in devito

In [17]:
#find the means
images, channels, height, width = batch_weights.shape
means = np.zeros(channels)
#838 µs for 7 runs, 1000 loops each
for channel in range(channels):
    for image in range(images):
        means[channel] += sum(sum(batch_weights[image][channel]))/(height*width)
means/=images
means


array([0.44230545, 0.40080094, 0.42683622])

In [20]:
E = torch.mean(batch_weights, axis = (0,2,3))
E
np.allclose(means, E)

True

In [22]:
#find std
var = np.zeros(channels)
for channel in range(channels):
    for image in range(images):
            var[channel] += sum(sum(pow(batch_weights[image][channel]-means[channel],2)))/(height*width)
var/=images
print(var)
Var = torch.var(batch_weights, unbiased=False, axis = (0,2,3))
print("is pytorch same as mine for Var?", np.allclose(Var, var))

[0.06291315 0.07446667 0.09313467]
is pytorch same as mine for Var? True


In [89]:
#batchnorm only substract with means
def batch_norm(weight):
    eps = 1e-5
    momentum = 0.1 #i dont know where it goes
    Var = torch.var(weight, unbiased=False, axis = (0,2,3))
    E = torch.mean(weight, axis = (0,2,3))
    print("E:", E, "Var",Var)
    gamma = 1
    betta = 0
    result = torch.zeros_like(batch_weights)
    for image in range(weight.shape[0]):
        for channel in range(weight.shape[1]):
            for x in range(weight.shape[2]):
                for y in range(weight.shape[3]):
                    result[image][channel][x][y]=(batch_weights[image][channel][x][y]-E[channel])\
                   /(np.sqrt(Var[channel]+1e-5))
    return result
py_res = batch_norm(batch_weights)

E: tensor([0.4423, 0.4008, 0.4268]) Var tensor([0.0629, 0.0745, 0.0931])


In [111]:
#devito
a, b, c, d = dimensions('a b c d')
gridB = Grid(shape=(2,3,3,3), dimensions = (a, b, c, d) )
B = Function(name='B', grid=gridB, space_order=0)

B.data[:] = batch_weights[:]

e, f, g, h = dimensions('e f g h')
gridR = Grid(shape=(2,3,3,3), dimensions = (e, f, g, h) )
R = Function(name='R', grid=gridR, space_order=0)

R.data[:] = 0

images, channels, height, width = batch_weights.shape
gridMean = Grid(shape=(channels))
M = Function(name= 'M', grid=gridMean,space_order=0)
#find the means

means = np.zeros(channels)
#838 µs for 7 runs, 1000 loops each
for channel in range(channels):
    for image in range(images):
        means[channel] += sum(sum(batch_weights[image][channel]))/(height*width)
means/=images
means
M.data[:]=means[:]
# var
gridVar = Grid(shape=(channels))
V = Function(name= 'V', grid=gridVar,space_order=0)
#find var

var = np.zeros(channels)
for channel in range(channels):
    for image in range(images):
            var[channel] += sum(sum(pow(batch_weights[image][channel]-means[channel],2)))/(height*width)
            
var/=images

#var = torch.var(batch_weights, axis = (0,2,3))
V.data[:] = var[:]

epsilon = 1e-5

#rhs = B[a,b,c,d]-M[b]
rhs = ((B[a,b,c,d]-M[b])/(((V[b]+epsilon)**(0.5))))

op = Operator(Eq(R[a, b, c, d], rhs))
op
print(op)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
} ;


int Kernel(struct dataobj *restrict B_vec, struct dataobj *restrict M_vec, struct dataobj *restrict R_vec, struct dataobj *restrict V_vec, const int a_M, const int a_m, const int b_M, const int b_m, const int c_M, const int c_m, const int d_M, const int d_m, struct profiler * timers, const int nthreads_nonaffine)
{
  float (*restrict B)[B_vec->size[1]][B_vec->size[2]][B_vec->size[3]] __attribute__ ((aligned (64))) = (float (*)[B_vec->size[1]][B_vec->size[2]][B_vec->size[3]]) B_vec->data;
  float (*restrict M) __attribute__ ((aligned (64))) = (float (*)) M_vec->data;
  float (*restrict R)[R_vec->size[1]][R_vec->size[2]][R_vec->size[3]] __attribute__ ((aligned (

In [116]:
summary = op.apply()
print(np.allclose(py_res, R.data))
py_res - R.data

Operator `Kernel` run in 0.01 s


True


tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 1.1921e-07,  1.1921e-07,  2.3842e-07],
          [ 1.0803e-07,  1.1921e-07,  5.9605e-08],
          [ 0.0000e+00,  1.1921e-07,  5.9605e-08]],

         [[ 1.1921e-07,  0.0000e+00,  0.0000e+00],
          [ 1.0431e-07,  5.9605e-08,  1.1921e-07],
          [ 1.1921e-07,  1.1921e-07,  5.9605e-08]]],


        [[[-1.1921e-07,  0.0000e+00,  0.0000e+00],
          [-5.9605e-08, -3.7253e-09,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 1.1921e-07,  1.1921e-07,  1.1921e-07],
          [ 1.1921e-07,  1.1921e-07,  1.1921e-07],
          [ 1.1921e-07,  1.1921e-07,  1.1921e-07]],

         [[ 5.9605e-08,  1.1921e-07,  9.4995e-08],
          [ 8.9407e-08,  5.9605e-08,  1.1921e-07],
          [ 1.1921e-07,  1.1921e-07,  1.1921e-07]]]])

In [92]:
py_res

tensor([[[[ 1.7517,  0.3583,  1.8362],
          [ 0.3391,  0.1500, -0.2117],
          [ 1.4750,  0.1713, -1.2843]],

         [[-1.1523, -1.2914,  2.1674],
          [-0.0621,  0.7287, -0.5442],
          [-1.2156,  1.4488, -0.6526]],

         [[-0.5391, -1.2252, -1.2412],
          [ 0.1044, -0.9599,  1.3165],
          [ 1.8083,  0.5404, -0.5330]]],


        [[[-1.7561, -0.6529,  0.6240],
          [-0.5909, -0.0582, -0.1333],
          [-1.1424,  0.4085, -1.2845]],

         [[ 0.1454, -1.0434, -0.5916],
          [ 0.6986, -0.6055,  0.3117],
          [ 0.2491, -0.2967,  1.7057]],

         [[-0.6627,  1.6172, -0.0307],
          [-0.4549, -0.9277,  0.7403],
          [-1.3396,  0.6794,  1.1076]]]])

In [109]:
batch_weights

tensor([[[[0.8817, 0.5322, 0.9029],
          [0.5274, 0.4799, 0.3892],
          [0.8123, 0.4853, 0.1202]],

         [[0.0863, 0.0484, 0.9923],
          [0.3839, 0.5997, 0.2523],
          [0.0691, 0.7962, 0.2227]],

         [[0.2623, 0.0529, 0.0480],
          [0.4587, 0.1339, 0.8286],
          [0.9787, 0.5918, 0.2642]]],


        [[[0.0018, 0.2785, 0.5988],
          [0.2941, 0.4277, 0.4089],
          [0.1557, 0.5448, 0.1201]],

         [[0.4405, 0.1160, 0.2394],
          [0.5915, 0.2356, 0.4859],
          [0.4688, 0.3198, 0.8663]],

         [[0.2246, 0.9204, 0.4175],
          [0.2880, 0.1437, 0.6528],
          [0.0180, 0.6342, 0.7649]]]])

In [42]:
torch_output - R.data

tensor([[[[ 1.3123,  0.2684,  1.3756],
          [ 0.2541,  0.1124, -0.1586],
          [ 1.1050,  0.1284, -0.9621]],

         [[-0.8378, -0.9389,  1.5759],
          [-0.0451,  0.5298, -0.3957],
          [-0.8838,  1.0534, -0.4745]],

         [[-0.3746, -0.8513, -0.8624],
          [ 0.0726, -0.6670,  0.9147],
          [ 1.2564,  0.3755, -0.3703]]],


        [[[-1.3156, -0.4891,  0.4675],
          [-0.4426, -0.0436, -0.0999],
          [-0.8559,  0.3060, -0.9623]],

         [[ 0.1057, -0.7587, -0.4301],
          [ 0.5080, -0.4402,  0.2266],
          [ 0.1811, -0.2157,  1.2402]],

         [[-0.4604,  1.1236, -0.0213],
          [-0.3161, -0.6446,  0.5144],
          [-0.9308,  0.4721,  0.7696]]]])

In [101]:
B.data

Data([[[[0.8817157 , 0.53217876, 0.90290296],
        [0.52737516, 0.47992754, 0.38920242],
        [0.8123122 , 0.4852845 , 0.12015229]],

       [[0.08633733, 0.04838318, 0.9923008 ],
        [0.38385695, 0.59966034, 0.2522865 ],
        [0.06906962, 0.7961829 , 0.22271138]],

       [[0.2622897 , 0.05289882, 0.04802912],
        [0.45870745, 0.13387042, 0.82863754],
        [0.9787218 , 0.59176695, 0.2641641 ]]],


      [[[0.00179935, 0.2785223 , 0.5988449 ],
        [0.29409194, 0.42771864, 0.40886676],
        [0.15573186, 0.5447799 , 0.1200909 ]],

       [[0.4404782 , 0.11603916, 0.23935169],
        [0.5914536 , 0.23556674, 0.48585397],
        [0.46877456, 0.31982666, 0.8662832 ]],

       [[0.22459686, 0.9203958 , 0.4174602 ],
        [0.28800595, 0.14369202, 0.6527707 ],
        [0.0179913 , 0.6341867 , 0.7648663 ]]]], dtype=float32)